In [1]:
import pandas as pd
import numpy as np
from session_rec.algorithms.knn import vsknn
from session_rec.evaluation import evaluation_last, evaluation
from session_rec.evaluation.metrics.accuracy import HitRate

/db_vol/aczapp/sandbox/recsys_eval_flaws/session_rec/evaluation/evaluation_last.py:74: SyntaxWarning: "is" with a literal. Did you mean "=="?
  last = next_session is -1 or next_session != sid


In [ ]:
metrics = [HitRate(1), HitRate(5), HitRate(10), HitRate(20)]
# best params can be found in "data/results/vskk_paropt/" after running vskkn_optimize.py
experiments_vsknn = {
    "full": {
        "train_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_train_full.tsv",
        "test_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_test.tsv",
        "best_params": {"k": 500, "sample_size": 1000, "weighting": "log", "weighting_score": "same", "idf_weighting": 2}},
    "91D": {
        "train_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_train_full_91D.tsv",
        "test_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_test.tsv",
        "best_params": {"k": 1000, "sample_size": 1000, "weighting": "log", "weighting_score": "same", "idf_weighting": 10}},
    "56D": {
        "train_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_train_full_56D.tsv",
        "test_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_test.tsv",
        "best_params": {"k": 500, "sample_size": 500, "weighting": "log", "weighting_score": "same", "idf_weighting": 5}},
    "28D": {
        "train_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_train_full_28D.tsv",
        "test_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_test.tsv",
        "best_params": {"k": 1000, "sample_size": 500, "weighting": "same", "weighting_score": "log", "idf_weighting": 5}},
    "14D": {
        "train_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_train_full_14D.tsv",
        "test_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_test.tsv",
        "best_params": {"k": 500, "sample_size": 500, "weighting": "same", "weighting_score": "log", "idf_weighting": 1}},
    "7D": {
        "train_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_train_full_7D.tsv",
        "test_file": "/db_vol/hb_work/rnn/data/public_raw_data/retailrocket_ecommerce/retailrocket_processed_view_7a_test.tsv",
        "best_params": {"k": 1500, "sample_size": 2500, "weighting": "log", "weighting_score": "same", "idf_weighting": 10}}
}